# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [10]:
# imports
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display

In [11]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [12]:
# set up environment
load_dotenv(override=True) #.env dosyasindaki degiskenleri (environment variables'e) yukler
api_key = os.getenv("OPENAI_API_KEY")

openai = OpenAI()

In [13]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

A system prompt that tells them what task they are performing and what tone they should use

A user prompt -- the conversation starter that they should reply to

In [14]:
# Get gpt-4o-mini to answer, with streaming
response = openai.chat.completions.create(model="gpt-4o-mini", messages = [{"role":"user","content" : question}])
print(response.choices[0].message.content) #role kimin konustugu content = mesajin icerigi
"system" → Asistanın kimliği veya davranışı. (Örn: “Sen bir Python öğretmenisin, kısa cevap ver”)
"user" → Kullanıcının gönderdiği mesajlar (senin yazdıkların).
"assistant" → Modelin (benim) verdiği önceki cevapla

This code snippet uses a generator expression along with the `yield from` statement. Let's break it down to understand what it does:

1. **Set Comprehension**: 
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   This part of the code creates a set of authors from a collection of `books`. It iterates through each `book` in the `books` iterable (which is likely a list of dictionaries). The expression `book.get("author")` retrieves the value associated with the "author" key for each book. The `if book.get("author")` condition ensures that only those books which actually have an author (i.e., the author value is not `None` or empty) are included in the set.

2. **`yield from` Statement**:
   ```python
   yield from ...
   ```
   The `yield from` statement is used in generator functions to yield all values from an iterable. By using `yield from`, the code effectively iterates over the entire set of authors created by the set comprehension and yields each 

In [17]:
# STREAM = yanıtın streaming (akışlı) modda mı yoksa batch (toplu) modda mı geleceğini belirler
# synchronous — tüm cevap gelene kadar bekler
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Sen kısa cevap veren bir Python öğretmenisin."},
        {"role": "user", "content": question}
    ]
)
print(response.choices[0].message.content)  # tüm cevap burada tek seferde


This code uses a generator expression with `yield from` to yield authors from a collection of `books`. Here's how it works:

1. It creates a set comprehension `{book.get("author") for book in books if book.get("author")}` that collects the unique authors from the `books` list, ensuring that only non-`None` author values are included.
2. `yield from` is used to yield each author from the set one at a time during iteration.

The use of a set comprehension ensures that duplicate authors are automatically removed, as sets only store unique values.


In [21]:
# Stream true olursa yaniti parca parca gonderir

import openai  # Önceki kodda import edilmiş varsayalım, yoksa ekle

# STREAM = True: Yanıt parça parça (token token) akar, döngüyle yakala
# Asynchronous hissi verir, ama synchronous olarak döngüde bekler (her parça geldiğinde yazdırır)
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Sen kısa cevap veren bir Python öğretmenisin."},
        {"role": "user", "content": question}
    ],
    stream=True  # Ana değişiklik: stream=True ekle
)

full_response = ""  # Opsiyonel: Tüm parçaları biriktir (eğer tam metin lazımsa)
for chunk in response:
    if chunk.choices[0].delta.content is not None:  # Delta: Yeni gelen parça
        content = chunk.choices[0].delta.content
        print(content, end='', flush=True)  # Parça parça ekrana bas (yazıyor efekti)
        full_response += content  # Biriktir (opsiyonel)

print()  # Sonunda yeni satıra geç

This code uses a generator expression with `yield from` to yield authors from a collection of books. Here's a breakdown:

1. **Set comprehension**: `{book.get("author") for book in books if book.get("author")}` creates a set of authors. It iterates through `books`, retrieves the value associated with the key `"author"` for each book, and includes it in the set only if the author exists (i.e., is not `None` or empty).

2. **`yield from`**: This keyword is used in a generator function to yield all values from the iterable (in this case, the set of authors) one by one.

So, the overall functionality is to create a generator that yields unique authors from the list of books, skipping any books that do not have an author.
